In [2]:
from glob import glob
from os import path

import numpy as np
import pandas as pd
import sklearn as sk


# [Dataset](https://drive.google.com/drive/folders/1nfrYxDm7TLzls9pedZbLX5rP4McVDWDe)

In [18]:
DATASET_PATH = "SSD2022AS2"


In [19]:
csv_files = glob(pathname=path.join(DATASET_PATH, "*.csv"), recursive=True)


In [20]:
dfs = list()

for csv_file in csv_files:
    dfs.append(pd.read_csv(filepath_or_buffer=csv_file))


In [43]:
df = pd.concat(objs=dfs).reset_index(drop=True)


In [ ]:
df.timestamp = pd.to_datetime(df.timestamp)


In [44]:
df.shape


(4035062, 8)

In [45]:
df.head()


,client_user_id,session_id,dropped_frames,FPS,bitrate,RTT,timestamp,device
0,0164608e-924e-407d-b7c6-f3c3fbaa882a,956d6b48-a337-41a5-8695-d265a6aa06ac,0.0,44.0,50.0,265.0,2022-09-14 08:31:10,Windows
1,0164608e-924e-407d-b7c6-f3c3fbaa882a,956d6b48-a337-41a5-8695-d265a6aa06ac,0.0,45.0,44.0,270.0,2022-09-14 08:31:15,Windows
2,0164608e-924e-407d-b7c6-f3c3fbaa882a,956d6b48-a337-41a5-8695-d265a6aa06ac,0.0,44.0,44.0,272.0,2022-09-14 08:31:20,Windows
3,0164608e-924e-407d-b7c6-f3c3fbaa882a,956d6b48-a337-41a5-8695-d265a6aa06ac,0.0,43.0,43.0,212.0,2022-09-14 08:31:25,Windows
4,0164608e-924e-407d-b7c6-f3c3fbaa882a,956d6b48-a337-41a5-8695-d265a6aa06ac,0.0,44.0,45.0,259.0,2022-09-14 08:31:30,Windows
...,...,...,...,...,...,...,...,...
4035057,fd68eb2f-98c9-4dc6-9724-1dde31ed5cbc,ddc6469e-874f-428d-9a58-6467ae07cd95,0.0,27.0,520.0,0.0,2022-09-19 20:21:35,Windows
4035058,fd68eb2f-98c9-4dc6-9724-1dde31ed5cbc,ddc6469e-874f-428d-9a58-6467ae07cd95,0.0,27.0,501.0,0.0,2022-09-19 20:21:40,Windows
4035059,fd68eb2f-98c9-4dc6-9724-1dde31ed5cbc,ddc6469e-874f-428d-9a58-6467ae07cd95,0.0,27.0,501.0,0.0,2022-09-19 20:21:45,Windows
4035060,fd68eb2f-98c9-4dc6-9724-1dde31ed5cbc,ddc6469e-874f-428d-9a58-6467ae07cd95,0.0,27.0,520.0,0.0,2022-09-19 20:21:50,Windows


In [ ]:
df.dtypes


# Stream Quality

In [46]:
df_s = (
    df.drop(labels=["timestamp", "device"], axis=1)
    .groupby(by=["client_user_id", "session_id"])
    .aggregate(
        {
            "dropped_frames": [np.mean, np.std, np.max],
            "FPS": [np.mean, np.std],
            "bitrate": [np.mean, np.std],
            "RTT": [np.mean, np.std],
        }
    )
)


In [47]:
df_s.head()

dropped_frames  \
                                                                                    mean   
client_user_id                       session_id                                            
003c2e7e-4392-4d01-81bb-93f2cce29f0b 8d39d204-d510-43cd-bf49-4df53f5ca0a5       9.166667   
004a4a88-03ec-4f2e-a0a3-9575fb7e3b1a 10a0bf75-0332-4fa9-8d5b-e2d90ec69bbf       0.510345   
006e829c-f656-4836-bc04-bafec3901fdc 11542f15-f5f2-4a02-9344-f0bbc8ad06ad       1.359472   
                                     38fd51c4-5e7c-4cd5-abcc-ff3d8e3c3180       1.276265   
                                     da0de052-ba6f-4f2c-8053-28d1cdcc0686       0.895911   

                                                                                      \
                                                                                 std   
client_user_id                       session_id                                        
003c2e7e-4392-4d01-81bb-93f2cce29f0b 8d39d204-d510-43cd-bf49-4df53f5ca0a5  27.891519   
004a4a88-03ec-4f2e-a0a3-9575fb7e3b1a 10a0bf75-0332-4fa9-8d5b-e2d90ec69bbf   6.243930   
006e829c-f656-4836-bc04-bafec3901fdc 11542f15-f5f2-4a02-9344-f0bbc8ad06ad   7.448480   
                                     38fd51c4-5e7c-4cd5-abcc-ff3d8e3c3180   8.476644   
                                     da0de052-ba6f-4f2c-8053-28d1cdcc0686   5.379302   

                                                                                  \
                                                                            amax   
client_user_id                       session_id                                    
003c2e7e-4392-4d01-81bb-93f2cce29f0b 8d39d204-d510-43cd-bf49-4df53f5ca0a5  159.0   
004a4a88-03ec-4f2e-a0a3-9575fb7e3b1a 10a0bf75-0332-4fa9-8d5b-e2d90ec69bbf   92.0   
006e829c-f656-4836-bc04-bafec3901fdc 11542f15-f5f2-4a02-9344-f0bbc8ad06ad   72.0   
                                     38fd51c4-5e7c-4cd5-abcc-ff3d8e3c3180   86.0   
                                     da0de052-ba6f-4f2c-8053-28d1cdcc0686   38.0   

                                                                                 FPS  \
                                                                                mean   
client_user_id                       session_id                                        
003c2e7e-4392-4d01-81bb-93f2cce29f0b 8d39d204-d510-43cd-bf49-4df53f5ca0a5  44.337719   
004a4a88-03ec-4f2e-a0a3-9575fb7e3b1a 10a0bf75-0332-4fa9-8d5b-e2d90ec69bbf  41.864368   
006e829c-f656-4836-bc04-bafec3901fdc 11542f15-f5f2-4a02-9344-f0bbc8ad06ad  29.026398   
                                     38fd51c4-5e7c-4cd5-abcc-ff3d8e3c3180  39.822957   
                                     da0de052-ba6f-4f2c-8053-28d1cdcc0686  26.847584   

                                                                                      \
                                                                                 std   
client_user_id                       session_id                                        
003c2e7e-4392-4d01-81bb-93f2cce29f0b 8d39d204-d510-43cd-bf49-4df53f5ca0a5  18.079243   
004a4a88-03ec-4f2e-a0a3-9575fb7e3b1a 10a0bf75-0332-4fa9-8d5b-e2d90ec69bbf  12.710460   
006e829c-f656-4836-bc04-bafec3901fdc 11542f15-f5f2-4a02-9344-f0bbc8ad06ad   4.702089   
                                     38fd51c4-5e7c-4cd5-abcc-ff3d8e3c3180   5.304531   
                                     da0de052-ba6f-4f2c-8053-28d1cdcc0686   4.424639   

                                                                                bitrate  \
                                                                                   mean   
client_user_id                       session_id                                           
003c2e7e-4392-4d01-81bb-93f2cce29f0b 8d39d204-d510-43cd-bf49-4df53f5ca0a5   3883.149123   
004a4a88-03ec-4f2e-a0a3-9575fb7e3b1a 10a0bf75-0332-4fa9-8d5b-e2d90ec69bbf   6415.157471   
006e829c-f656-4836-bc04-bafec3901fdc 11542f15-f5f2-4a02-9344-f0bbc8ad06ad  17226.447205   
   

In [48]:
df_s.columns = ["_".join(column).lower() for column in df_s.columns.to_flat_index()]


In [49]:
df_s = df_s.reset_index(drop=True)


In [50]:
df_s.head()


,dropped_frames_mean,dropped_frames_std,dropped_frames_amax,fps_mean,fps_std,bitrate_mean,bitrate_std,rtt_mean,rtt_std
0,9.166667,27.891519,159.0,44.337719,18.079243,3883.149123,5381.456809,171.109649,277.138638
1,0.510345,6.243930,92.0,41.864368,12.710460,6415.157471,4366.984310,27.934483,11.900105
2,1.359472,7.448480,72.0,29.026398,4.702089,17226.447205,5076.756036,58.000776,9.220387
3,1.276265,8.476644,86.0,39.822957,5.304531,3930.153696,1532.691830,54.888132,55.603848
4,0.895911,5.379302,38.0,26.847584,4.424639,13372.773234,4410.790738,16.405204,8.144871


In [51]:
df_s.to_csv(path_or_buf="output_stream.csv")


# Next Session Time

In [30]:
df_t = df.groupby(by=["client_user_id", "session_id"]).aggregate(
    {
        "dropped_frames": [np.mean, np.std, np.max],
        "FPS": [np.mean, np.std],
        "bitrate": [np.mean, np.std],
        "RTT": [np.mean, np.std],
        "timestamp": [np.ptp],
    }
)


In [32]:
df_t.head()


dropped_frames  \
                                                                                    mean   
client_user_id                       session_id                                            
003c2e7e-4392-4d01-81bb-93f2cce29f0b 8d39d204-d510-43cd-bf49-4df53f5ca0a5       9.166667   
004a4a88-03ec-4f2e-a0a3-9575fb7e3b1a 10a0bf75-0332-4fa9-8d5b-e2d90ec69bbf       0.510345   
006e829c-f656-4836-bc04-bafec3901fdc 11542f15-f5f2-4a02-9344-f0bbc8ad06ad       1.359472   
                                     38fd51c4-5e7c-4cd5-abcc-ff3d8e3c3180       1.276265   
                                     da0de052-ba6f-4f2c-8053-28d1cdcc0686       0.895911   
...                                                                                  ...   
ff5c29dc-94cf-4e36-8550-d9a2583ca439 ae9d4b9f-66fe-432d-8f80-0f32932a5be0       1.073446   
                                     c2ae6769-20a1-4086-817a-877afe436855       0.488956   
ff6840c7-06cc-4b09-89d9-fb680aed5385 641534d9-3328-49d3-b411-fa3ac736045b       0.516746   
ff88bb69-10c4-41d5-83a7-96663f165c33 2a296f3e-d81f-475d-9d35-6843f91b6e3d       0.000000   
ffbeddd5-ca3f-4bda-9e93-8b3e9c792bd4 795832de-87ac-48db-a738-c77842d26338      24.102041   

                                                                                      \
                                                                                 std   
client_user_id                       session_id                                        
003c2e7e-4392-4d01-81bb-93f2cce29f0b 8d39d204-d510-43cd-bf49-4df53f5ca0a5  27.891519   
004a4a88-03ec-4f2e-a0a3-9575fb7e3b1a 10a0bf75-0332-4fa9-8d5b-e2d90ec69bbf   6.243930   
006e829c-f656-4836-bc04-bafec3901fdc 11542f15-f5f2-4a02-9344-f0bbc8ad06ad   7.448480   
                                     38fd51c4-5e7c-4cd5-abcc-ff3d8e3c3180   8.476644   
                                     da0de052-ba6f-4f2c-8053-28d1cdcc0686   5.379302   
...                                                                              ...   
ff5c29dc-94cf-4e36-8550-d9a2583ca439 ae9d4b9f-66fe-432d-8f80-0f32932a5be0   9.303185   
                                     c2ae6769-20a1-4086-817a-877afe436855   5.222764   
ff6840c7-06cc-4b09-89d9-fb680aed5385 641534d9-3328-49d3-b411-fa3ac736045b   5.284316   
ff88bb69-10c4-41d5-83a7-96663f165c33 2a296f3e-d81f-475d-9d35-6843f91b6e3d   0.000000   
ffbeddd5-ca3f-4bda-9e93-8b3e9c792bd4 795832de-87ac-48db-a738-c77842d26338  26.853961   

                                                                                  \
                                                                            amax   
client_user_id                       session_id                                    
003c2e7e-4392-4d01-81bb-93f2cce29f0b 8d39d204-d510-43cd-bf49-4df53f5ca0a5  159.0   
004a4a88-03ec-4f2e-a0a3-9575fb7e3b1a 10a0bf75-0332-4fa9-8d5b-e2d90ec69bbf   92.0   
006e829c-f656-4836-bc04-bafec3901fdc 11542f15-f5f2-4a02-9344-f0bbc8ad06ad   72.0   
                                     38fd51c4-5e7c-4cd5-abcc-ff3d8e3c3180   86.0   
                                     da0de052-ba6f-4f2c-8053-28d1cdcc0686   38.0   
...                                                                          ...   
ff5c29dc-94cf-4e36-8550-d9a2583ca439 ae9d4b9f-66fe-432d-8f80-0f32932a5be0  148.0   
                                     c2ae6769-20a1-4086-817a-877afe436855   75.0   
ff6840c7-06cc-4b09-89d9-fb680aed5385 641534d9-3328-49d3-b411-fa3ac736045b   58.0   
ff88bb69-10c4-41d5-83a7-96663f165c33 2a296f3e-d81f-475d-9d35-6843f91b6e3d    0.0   
ffbeddd5-ca3f-4bda-9e93-8b3e9c792bd4 795832de-87ac-48db-a738-c77842d26338   69.0   

                                                                                 FPS  \
                                                                                mean   
client_user_id                       session_id                                        
003c2e7e-4392-4d01-81bb-93f2cce29f0b 8d39d204-d510-43cd-bf49-4df53f5ca0a5  44.337719   
004a4a88-03ec-4f2e-a0a3

In [ ]:
df_t.columns = ["_".join(column).lower() for column in df_t.columns.to_flat_index()]


In [ ]:
df_t = df_t.reset_index(drop=True)


In [ ]:
df_t.head()


In [ ]:
df_t.to_csv(path_or_buf="output_time.csv")


# Data Engineering

# Total Number of Bad Sessions